# A statistical Conllu file Exploration of  Universal Dependencies

## Introduction

While much work is being done in the current days on NLP and NLU, there is little work on describing why a certain length of transformer (or other as LSTM time steps) architecture has been chosen for the training, it is mostly arbitrary and depends on the goal of the work and resources available (mainly hardware). These decisions are hard once the model has been trained and there is nothing that can be done to extend the length of a transformer (for example) without having to retrain the entire network. There are however some works that tackle variable length sequences. 

This work presents the first complete analysis (from our knowledge) of the Universal Dependencies v2.6 dataset and presents the global and individual results by language.

This work does not intend to be a conference level paper (that is why there are no references to papers on the subject), but an informational technical report that might help to better select the most effective compromise text or token length for your particular NLP application.

The number of analyzed languages is 91 (out of 92), the token length is measured as the named UPOS tag in the dataset, while the character length is just that. There is no analysis on what constitutes a word or not, this means that a token includes the punctuation and other symbols presents in the text samples.



## Dataset

[Universal Dependencies](https://universaldependencies.org/) [v2.6](https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-3226)

The process is the same for any UD version.

The Data is **353MB** compressed in gzip format and **2.1GB** in uncompressed raw text format.

The entire pipeline takes about 17-18 minutes to process and generate all graphs and html output from the uncompressed UD dataset on 1 CPU core of an intel 7700.


## Technology Stack

The analysis was done with the following software stack:
* Ubuntu 20.04
* python 3.8 with all dependencies installed in a virtualenv 
* conllu files are dealt with [pyconll](https://github.com/pyconll/pyconll)
* [pycountry  v19.8.18](https://pypi.org/project/pycountry/) is used to transform between country code and country names
* [NumPy v1.18.4](https://numpy.org/) is used for numerical computation
* [Pandas v1.0.4](https://pandas.pydata.org/) is used for DataFrame generation and making easier some operations
* [SciPy v1.4.1](https://www.scipy.org/) is used for statistics
* [Matplotlib v3.2.1](https://matplotlib.org/) was used for the first graphing iterations during the exploration stage
* [Bokeh v2.0.2](https://docs.bokeh.org/en/latest/index.html) used for the latest graph and table generation to make interactive presentations
* [Jupyter Lab v2.1.2](https://jupyterlab.readthedocs.io/en/stable/) was used during the entire process from starting the exploration to writing the current report

## Method - Steps

The code detail can be seen at the end of this document, after the conclussions

1. A single zipped file containing the dataset was downloaded locally and uncompressed
2. All the file tree is explored, a list of all the files is returned. All files, train, test and dev are used for this analysis to have the maximum amount of data available.
3. undesired files are filtered out (used only during exploration to select different languages to explore and check errors)
4.  each file:
 1. Open the file
 2. language code (2 or 3 letter) is extracted from the file name
 3. sentence, form and upos fields are extracted for each file
5. All the separate sentence, upos and forms data from each files are now merged in a single list for each type and enriched with language information
6. Data is now set in a Pandas Dataframe to simplify handling (select by language in this case). For development or different needs a list of languages can be given to accelerate the processing (for example for exploration purposes)
7. Every language is now processed independently and several possible distributions are tested, only the one with the best Kolmogorov-Smirnov test result is chosen as a result. This is the most time-expensive step in the pipe.
8. Statistics are computed for every language
9. Results are converted to json and saved in a single gz file (available to download in the github repository)
10. Statistics tables and plots are done per language, graphs can be explored in the [project's github page](TODO)
11. Tables are created for upos and text (character) length for all the languages together. These tables are available in the  [project's github page](TODO)

## Observations

The histograms show a skew on the distribution, this can be a skewed gaussian, a generalized gaussian or a beta distribution form. Due to this, I will be testing different distribution fits with the Kolmogorov-Smirnov test.

There are many languages that do not have enough samples so the dsitribution fit will not be good  and errors will be big.
This is not an issue  from the code point of view. The important thing is if this data is used, take into account the number of samples available.


While doing this work I found quite interesting that are languages whose number of tokens or characters avoid certain bins in the histogram (Bulgarian, Breton Welsh, Danish, Slovak, Tamil and Thai are a few examples of this). This can mean that, either the language structure supports only those lengths, or that the analyzed dataset only contains samples that avoid some sentence lengths.

For some languages the number of samples is too small to make any good assumption from the data.

There is one missing language in the analysis, TODO HERE ...



## Conclusion

This work presents a sample length analysis by language on the UniversalDependencies v2.6 dataset presenting the statistics for all 91 of the 92 represented languages. The analysis then shows the length histograms by character and token length.

The best compromise for choosing a sequence length on the NLP architecture for training will depend mostly on the requirements of the application, nevertheless with the numbers here you should be able to make an informed guess on what might be better for your case.

We can see that having a multi-lingual approach will necessary make the needed sequences longer as there is a large variability on sequence length, but appliying to single language might allow you to optimize your neural architectures.

## Future Work

We are currently working on a more in depth analysis of the complete Gutenberg project dataset ( ~60K books in several languages) that will discriminate several other text characteristics.

Stay tuned for those results ;)

In [1]:
from preprocessors.ud_conllu_stats import *
import preprocessors.ud_conllu_stats as udstats
from bokeh.models import Dropdown
from bokeh.models.layouts import Column, Panel, Tabs
from bokeh.models.callbacks import CustomJS
from bokeh.io import output_notebook


In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
from ipywidgets import interact, interact_manual

In [4]:
%%time
all_stats = generate_files()

/home/leo/venv3/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:547: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/leo/venv3/lib/python3.8/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/home/leo/venv3/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:708: RuntimeWarning: divide by zero encountered in true_divide
  a/(b-1.0),
/home/leo/venv3/lib/python3.8/site-packages/scipy/stats/_continuous_distns.py:712: RuntimeWarning: divide by zero encountered in true_divide
  a*(a+1.0)/((b-2.0)*(b-1.0)),
/home/leo/venv3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1063: RuntimeWarning: invalid value encountered in subtract
  mu2 = mu2p - mu * mu
/home/leo/venv3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.p

Error processing lang qhe with Exception 'NoneType' object has no attribute 'name'
processing af
processing aii
processing akk
processing am
processing ar
processing be
processing bg
processing bho
processing bm
processing br
processing bxr
processing ca
processing cop
processing cs
processing cu
processing cy
processing da
processing de
processing el
processing en
processing es
processing et
processing eu
processing fa
processing fi
processing fo
processing fr
processing fro
processing ga
processing gd
processing gl
processing got
processing grc
processing gsw
processing gun
processing he
processing hi
processing hr
processing hsb
processing hu
processing hy
processing id
processing is
processing it
processing ja
processing kk
processing kmr
processing ko
processing koi
processing kpv
processing krl
processing la
processing lt
processing lv
processing lzh
processing mdf
processing mr
processing mt
processing myv
processing nl
processing no
processing olo
processing orv
processing pcm


In [5]:
%%time

all_stats_copy = copy.deepcopy(all_stats)
grids = udstats._generate_html_plots(all_stats_copy)

CPU times: user 5.89 s, sys: 16 ms, total: 5.9 s
Wall time: 5.9 s


In [6]:
%%time
all_stats_copy = copy.deepcopy(all_stats)
upos_table, text_table = udstats._make_complete_stats_tables(all_stats_copy)

CPU times: user 47.4 ms, sys: 0 ns, total: 47.4 ms
Wall time: 47.4 ms


In [7]:
%%time
upos_html = udstats._generate_html_table(upos_table, 'upos_table')

CPU times: user 13.1 ms, sys: 0 ns, total: 13.1 ms
Wall time: 12.7 ms


In [8]:
%%time
text_html = udstats._generate_html_table(text_table, 'text_table')

CPU times: user 14.5 ms, sys: 7 µs, total: 14.5 ms
Wall time: 14.2 ms


In [9]:
lang_menu = [(code, (languages.get(alpha_2=code) if len(code) == 2 else languages.get(alpha_3=code)).name) for code in grids[0].keys()]

In [10]:
lang_menu

[('af', 'Afrikaans'),
 ('aii', 'Assyrian Neo-Aramaic'),
 ('akk', 'Akkadian'),
 ('am', 'Amharic'),
 ('ar', 'Arabic'),
 ('be', 'Belarusian'),
 ('bg', 'Bulgarian'),
 ('bho', 'Bhojpuri'),
 ('bm', 'Bambara'),
 ('br', 'Breton'),
 ('bxr', 'Russia Buriat'),
 ('ca', 'Catalan'),
 ('cop', 'Coptic'),
 ('cs', 'Czech'),
 ('cu', 'Church Slavic'),
 ('cy', 'Welsh'),
 ('da', 'Danish'),
 ('de', 'German'),
 ('el', 'Modern Greek (1453-)'),
 ('en', 'English'),
 ('es', 'Spanish'),
 ('et', 'Estonian'),
 ('eu', 'Basque'),
 ('fa', 'Persian'),
 ('fi', 'Finnish'),
 ('fo', 'Faroese'),
 ('fr', 'French'),
 ('fro', 'Old French (842-ca. 1400)'),
 ('ga', 'Irish'),
 ('gd', 'Scottish Gaelic'),
 ('gl', 'Galician'),
 ('got', 'Gothic'),
 ('grc', 'Ancient Greek (to 1453)'),
 ('gsw', 'Swiss German'),
 ('gun', 'Mbyá Guaraní'),
 ('he', 'Hebrew'),
 ('hi', 'Hindi'),
 ('hr', 'Croatian'),
 ('hsb', 'Upper Sorbian'),
 ('hu', 'Hungarian'),
 ('hy', 'Armenian'),
 ('id', 'Indonesian'),
 ('is', 'Icelandic'),
 ('it', 'Italian'),
 ('ja', 'J

In [11]:
# generate html options for web page
optlines = []
count = 1

for code,lang in lang_menu:
    opt = '[{}] - {}'.format(code, lang)
    if code == 'fr':
        line = '<option value="{}" selected="selected">{}</option>\n'.format(code, opt)
    else:
        line = '<option value="{}">{}</option>\n'.format(code, opt)
    optlines.append(line)
    count += 1

In [12]:
f = open('lang_menu.html', 'w')
f.writelines(optlines)
f.flush()
f.close()

In [13]:

# all_stats_source = 
# all_quantiles_source = 
# all_graph_sources = 
# all_titles_source = 


In [14]:

# source_stats = 
# source_quantiles = 
# source_graph = 
# source_title = 


In [15]:
# # output_file("dropdown.html")

# dropdown = Dropdown(label="Dropdown button", menu=lang_menu)

# callback_lang_select = CustomJS(args=dict(), code="""
#     var data = source.data;
#     data = all_data[value];
#     title = all_data;
#     source.change.emit();
# """)

# dropdown.js_on_event('value', callback_lang_select)
# # show(dropdown)

# dd_comp = components(dropdown)


In [16]:
# dd_comp

In [17]:
# output_file("slider.html")

# charlen_panel = Panel(child=text_table, title="Character Length - all languages")
# upos_panel = Panel(child=upos_table, title="Token Length - all languages")

In [18]:
# del (charlen_panel)
# del(upos_panel)


In [19]:
# grid = 

In [20]:
# gp_col = Column(dropdown)

In [21]:
# graph_children = grids
# graph_panel = Panel(child=gp_col)

In [22]:
# tab1 = Panel(child=text_table, title="Characters Table")
# tab2 = Panel(child=upos_table, title="Token Table")
# tab3 = Panel(child=gp_col, title="per language graphs")

# # tabs = Tabs(tabs=[ tab1, tab2])
# tabs = Tabs(tabs=[ tab1, tab2, tab3 ])

In [23]:
# show(tabs)